In [ ]:
# Standard library imports
import subprocess
import sys
from pathlib import Path

from multiview_mapping_toolkit.utils.visualization import show_segmentation_labels
from multiview_mapping_toolkit.cameras.derived_cameras import MetashapeCameraSet

# Imports from the constants
sys.path.append("../..")
from constants import (
    MMSEG_PYTHON,
    MMSEG_UTILS_PYTHON,
    INFERENCE_SCRIPT,
    VIS_PREDS_SCRIPT,
    LABELS_FILENAME,
    get_inference_image_folder,
    get_work_dir,
    get_prediction_folder,
    get_images_near_labels_folder,
    get_image_folder,
    get_oblique_images_folder,
    get_camera_filename
)

# Set constants
You should be able to define most of the behavior from these constants

In [ ]:
TEST_SITE = "none"
TRAINING_SITES = sorted(["none", "none", "none"])

BUFFER_RADIUS_METERS = 50

In [9]:
IMAGE_FOLDER = get_image_folder(TEST_SITE)
IMAGE_FOLDER_OBLIQUE = get_oblique_images_folder(TEST_SITE)
CAMERAS_FILENAME = get_camera_filename(TEST_SITE)

# Create camera set
camera_set = MetashapeCameraSet(CAMERAS_FILENAME, IMAGE_FOLDER)
# Extract cameras near the training data
training_camera_set = camera_set.get_subset_ROI(
    ROI=LABELS_FILENAME, buffer_radius_meters=BUFFER_RADIUS_METERS
)
training_camera_set = training_camera_set.get_cameras_in_folder(IMAGE_FOLDER_OBLIQUE)
training_camera_set.vis(force_xvfb=True, frustum_scale=0.5)


IMAGES_NEAR_LABELS_FOLDER = get_images_near_labels_folder(TEST_SITE)

print(f"Saving subset of images near labels to {IMAGES_NEAR_LABELS_FOLDER}")
training_camera_set.save_images(IMAGES_NEAR_LABELS_FOLDER)

# Generate predictions with the model

In [ ]:
WORK_DIR = get_work_dir(training_sites=TRAINING_SITES, is_ortho=False, is_scratch=False)
INFERENCE_IMAGE_FOLDER = get_inference_image_folder(site_name=TEST_SITE)
PREDICTIONS_FOLDER = get_prediction_folder(
    prediction_site=TEST_SITE, training_sites=TRAINING_SITES, is_ortho=False
)


config_file = list(Path(WORK_DIR).glob("*py"))[0]
checkpoint_file = Path(WORK_DIR, "iter_10000.pth")
BATCH_SIZE = 2

subprocess.run(
    f"{MMSEG_PYTHON} {INFERENCE_SCRIPT} {config_file} "
    + f"{checkpoint_file} {INFERENCE_IMAGE_FOLDER} {PREDICTIONS_FOLDER} --batch-size {BATCH_SIZE}",
    shell=True,
)

# Visualize predictions

In [ ]:
# Add _vis to the folder name. Can't just add _vis to the str representation because it might have a trailing slash
pred_vis_dir = Path(
    Path(PREDICTIONS_FOLDER).parent, Path(PREDICTIONS_FOLDER).parts[-1] + "_vis"
)
STRIDE = 10

subprocess.run(
    f"{MMSEG_UTILS_PYTHON} {VIS_PREDS_SCRIPT} --image-dir {INFERENCE_IMAGE_FOLDER} "
    + f"--seg-dir {PREDICTIONS_FOLDER} --output-dir {pred_vis_dir} --stride {STRIDE}",
    shell=True,
)

In [ ]:
show_segmentation_labels(
    label_folder=PREDICTIONS_FOLDER, image_folder=INFERENCE_IMAGE_FOLDER
)